__<font size="6" weight="bold">Bayes Trainer for Validator</font>__
<br>
<strong>Not for production!</strong>
Hier werden die Modelle für den Bayes angelernt. in einem Durchgang werden insgesamt 4 verschiedene Modelle trainiert: 
<br><br>Nicht lemmatisiert und keine Stoppwörter entfernt
<br> Nicht lemmatisiert und Stoppwörter entfernt
<br> lemmatisiert und keine Stoppwörter entfernt
<br> lemmatisiert und  Stoppwörter entfernt<br>
<br>
Dazu werden alle Pattern durchlaufen. Für jedes Pattern wird ein DataFrame mit positiven und negativen Sätzen erstellt. Dieser Dataframe wird dann in X Test und Y Test aufgeteilt. Hierbei werden immer die ersten 85% fürs training genutzt und die restlichen 15% für das Testen. Damit kann sicher gestellt werden, dass alle Modelle identisch sind von den Daten.
<br>
Für jedes Modell wird ein eigener Ordner angelegt.

## Initialisieren der benötigten Klassen und Libraries

In [1]:
import spacy
import pickle
import os 
import os.path as path
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords

In [2]:
%run REST.ipynb
%run TrainingSetFactory_for_BayesTrainer_lemma_no_lemma_stopwords_no_stopwords.ipynb

## Bayes Trainer not for production only for testing.


In [3]:
class BayesTrainer: 
    directoryname="Bayes"
    directoryname_output ="Bayes_Output"

    def train(self, id, dataframe):        
        #create directory if not exist
        os.makedirs(self.directoryname, exist_ok=True)
        os.makedirs(self.directoryname+"/" +self.directoryname_output, exist_ok=True)
        bayes_file = self.get_filename(id,self.directoryname,"Bayes")
        countervector_file = self.get_filename(id,self.directoryname,"Counter")
        X = dataframe[0]
        Y = dataframe[1]
        # random_state = 0 to ensure that every instance has the same training and test data
        X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.85, random_state=0)
        naive_bayes = BernoulliNB()
        #wordvetor erstellung
        vectorizer = CountVectorizer(binary=True)
        x_train = vectorizer.fit_transform(X_train)
        x_test = vectorizer.transform(X_test)
        pickle.dump(vectorizer,open(countervector_file,"wb"))
        #trainieren
        naive_bayes.fit(x_train, y_train)
        predictions = naive_bayes.predict(x_test)
        f = open(self.directoryname+"/"+self.directoryname_output+ "/Precision_Accuracy_Recall_F1_:"+str(id)+".txt","w")
        f.write("Accuracy score"+ str (accuracy_score(y_test, predictions))+"\n")
        f.write('Precision score: ' + str (precision_score(y_test, predictions))+"\n")
        f.write('Recall score: ' + str (recall_score(y_test, predictions))+"\n")
        f.write('f1_score score: ' + str (f1_score(y_test, predictions))+"\n")
        f.close()
        pickle.dump(naive_bayes, open(bayes_file, 'wb'))
        
    def get_filename(self, id,filepath,name=""):
        filename = "/" + name +"-"+str(id)+".sav"
        return filepath + filename

In [4]:
client = empamos_rest_client()

## Test Stoppwörter/Lemmatisieren Stoppwörter entfernt/Lemmatisiert,Stoppwörter entfernt/Nicht Lemmatisiert,Stoppwörter/Lemmatisiert



In [5]:
loc = BayesTrainer()
factory = TrainingSetFactory(4)
pattern_ids = [pattern['pattern_id'] for pattern in client.get_patterns()]

for id in pattern_ids:
    df = factory.trainings_dataframe(id) 
    if (factory.isValid):
        loc.directoryname = "Lemmatized_With_Stopwords"
        #
        dfPos = factory.Sentences_to_DF(factory.tokenized_pos_sentences,1)
        dfNeg = factory.Sentences_to_DF(factory.tokenized_neg_sentences,0)
        #
        df = dfPos.append(dfNeg)
        loc.train(id, df)
        f = open("Lemmatized_With_Stopwords/"+str(id)+"Sätze.csv","w")
        df.to_csv(f, header=False)
        f.close()       
        ###
        loc.directoryname = "Bayes_without_stopwords_lemmatized"
        dfPos = factory.Sentences_to_DF(factory.tokenized_stopwords_removed_pos_sentences,1)
        dfNeg = factory.Sentences_to_DF(factory.tokenized_stopwords_removed_neg_sentences,0)
        #
        df = dfPos.append(dfNeg)     
        loc.train(id, df)
        f = open("Bayes_without_stopwords_lemmatized/"+str(id)+"Sätze.csv","w")
        df.to_csv(f, header=False)
        f.close()          
        ### 
        loc.directoryname = "Bayes_no_lemmatize_with_stopwords"
        dfPos = factory.Sentences_to_DF(factory.no_tokenized_with_stopwords_pos_sentences,1)
        dfNeg = factory.Sentences_to_DF(factory.no_tokenized_with_stopwords_neg_sentences,0)
        #
        df = dfPos.append(dfNeg)
        # 
        loc.train(id, df)
        f = open("Bayes_no_lemmatize_with_stopwords/"+str(id)+"Sätze.csv","w")
        df.to_csv(f, header=False)
        f.close()      
        #
        loc.directoryname = "Bayes_no_lemmatize_stopword_removed"
        #
        dfPos = factory.Sentences_to_DF(factory.no_tokenized_removed_stopwords_pos_sentences,1)
        dfNeg = factory.Sentences_to_DF(factory.no_tokenized_removed_stopwords_neg_sentences,0)
        #
        df = dfPos.append(dfNeg)     
        loc.train(id, df)  
        f = open("Bayes_no_lemmatize_stopword_removed/"+str(id)+"Sätze.csv","w")
        df.to_csv(f, header=False)
        f.close()  
        print("trained!"+str(id))
        
    else:
        print(f'Keine Daten vorhanden! {id}')


        
        
        

    

trained!1292
Keine Daten vorhanden! 1409
Keine Daten vorhanden! 1417
Keine Daten vorhanden! 1433
Keine Daten vorhanden! 1446
Keine Daten vorhanden! 1455
Keine Daten vorhanden! 1464
Keine Daten vorhanden! 1473
Keine Daten vorhanden! 1482
trained!1492
Keine Daten vorhanden! 1515
Keine Daten vorhanden! 1665
Keine Daten vorhanden! 1756
Keine Daten vorhanden! 2884
Keine Daten vorhanden! 2894
Keine Daten vorhanden! 2907
Keine Daten vorhanden! 2968
Keine Daten vorhanden! 2981
Keine Daten vorhanden! 3085
Keine Daten vorhanden! 3098
trained!3142
trained!3173
Keine Daten vorhanden! 3194
Keine Daten vorhanden! 3285
Keine Daten vorhanden! 3312
Keine Daten vorhanden! 3377
Keine Daten vorhanden! 3755
Keine Daten vorhanden! 4426
Keine Daten vorhanden! 5718
Keine Daten vorhanden! 5729
Keine Daten vorhanden! 5740
trained!7090
Keine Daten vorhanden! 7098
Keine Daten vorhanden! 7106
Keine Daten vorhanden! 7115
Keine Daten vorhanden! 7123
Keine Daten vorhanden! 7227
Keine Daten vorhanden! 8520
trained!852